本notebook的CNN模型是只是在基本的CNN网络上加了一层嵌入结构

## 1、导入相关包

In [1]:
import os
from tqdm import tqdm
# 数据科学包
import random                      # 随机切分数据集
import numpy as np                 # 常用数据科学包
import pandas as pd              # 图像读取
import matplotlib.pyplot as plt    # 代码中快速验证
import cv2                         # 图像包

# 深度学习包
import paddle
import paddle.vision.transforms as tf      # 数据增强
from paddle.io import Dataset, DataLoader  # 定义数据集
import paddle.nn as nn                     # 网络

/root/miniconda3/envs/paddle/lib/python3.8/site-packages/setuptools/sandbox.py:13: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/root/miniconda3/envs/paddle/lib/python3.8/site-packages/pkg_resources/__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/root/miniconda3/envs/paddle/lib/python3.8/site-packages/pkg_resources/__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#key

## 2、准备数据集

处理数据集

## 3、准备网络模型

搭建网络

In [2]:
# 定义CNN模型：随机嵌入
class ModifiedCNN(nn.Layer):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, kernel_size, output_size, maxlength):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.cnn = nn.Conv1D(embedding_dim, hidden_dim, kernel_size)
        self.maxpool = nn.MaxPool1D(maxlength - kernel_size + 1)
        self.bi_lstm = nn.LSTM(hidden_dim, hidden_dim // 2, num_layers=1, direction='bidirectional')
        self.dense = nn.Sequential(nn.Dropout(0.3), nn.Linear(hidden_dim, output_size))

    def forward(self, x):
        embed_x = self.embed(x)
        cnn_x = self.cnn(embed_x.transpose((0, 2, 1)))
        pool_x = self.maxpool(cnn_x)
        lstm_out, _ = self.bi_lstm(pool_x.squeeze(-1).transpose((0, 2, 1)))  # Transpose input for PaddlePaddle LSTM
        out = self.dense(lstm_out[:, -1, :])  # Take the output of the last time step
        return out


# 定义模型
vocab_size = 10000 # 词汇数量
embedding_dim = 1024 # 词嵌入维度
hidden_dim = 128 # 隐藏层维度，也就是CNN网络层卷积核的个数
kernel_size = 3 # 卷积核大小
output_size = 14  # 分类的类别数
maxlength = 30  # 新闻标题的最大长度

model = ModifiedCNN(vocab_size, embedding_dim, hidden_dim, kernel_size, output_size, maxlength)


W1229 12:32:18.332525 74843 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 8.0, Driver API Version: 12.2, Runtime API Version: 11.7
W1229 12:32:18.340533 74843 gpu_resources.cc:91] device: 0, cuDNN Version: 8.5.


查看网络结构

In [3]:
# 输出模型结构
print(model)


ModifiedCNN(
  (embed): Embedding(10000, 1024, padding_idx=0, sparse=False)
  (cnn): Conv1D(1024, 128, kernel_size=[3], data_format=NCL)
  (maxpool): MaxPool1D(kernel_size=28, stride=None, padding=0)
  (bi_lstm): LSTM(128, 64
    (0): BiRNN(
      (cell_fw): LSTMCell(128, 64)
      (cell_bw): LSTMCell(128, 64)
    )
  )
  (dense): Sequential(
    (0): Dropout(p=0.3, axis=None, mode=upscale_in_train)
    (1): Linear(in_features=128, out_features=14, dtype=float32)
  )
)


## 4、模型训练

## 5、模型测试